In [ ]:
# Blocks opponents on depth 1(2)

def my_agent(observation, configuration):
    
    # Number of Columns on the Board.
    columns = configuration.columns
    # Number of Rows on the Board.
    rows = configuration.rows
    # Number of Checkers "in a row" needed to win.
    inarow = configuration.inarow
    # The current serialized Board (rows x columns).
    board = observation.board
    # Which player the agent is playing as (1 or 2).
    mark = observation.mark
    
    #returns cheker on boor coordinate. Top left coord is (0, 0)
    def get_checker_by_coord(x, y, observation, configuration):
        return observation.board[x + (configuration.columns * y)]
        
    #returns the most downward row in a columns
    def find_empty_row_in_column(x, observation, configuration):
        column = get_column(x, observation, configuration)
        y = -1
        coord = 0
        for checker in column:
            if checker == 0: 
                y = coord
                coord += 1
        return y
    
    #returns a column by X coordinate. Left X = 0
    def get_column(x, observation, configuration):
        column = []
        for y in range(0 ,configuration.rows):
            checker = get_checker_by_coord(x, y, observation, configuration)
            column.append(checker)
        return column
    
    #returns a row by Y coordinate. Top Y = 0
    def get_row(y, observation, configuration):
        row = []
        for x in range(0,configuration.columns):
            checker = get_checker_by_coord(x, y, observation, configuration)
            row.append(checker)
        return row
    
    #returns a diagonal (left_top -> bottom_right), inputs coordinates should be in 'Top row' or 'Left collumn'
    def get_main_diagonal(x, y, observation, configuration):
        diag = []
        if x * y != 0:
            raise Exception('Incorrect coordinates: either x or y should be 0')
        while x <= configuration.columns -1 and y <= configuration.rows - 1:
            checker = get_checker_by_coord(x, y, observation, configuration)
            diag.append(checker)
            x += 1
            y += 1
        return diag
    
    #returns a diagonal (left_bot -> top_right), inputs coordinates should be in 'Bottom row' or 'Left collumn'
    def get_secondary_diagonal(x, y, observation, configuration):
        diag = []
        if not (x == 0 or y == (configuration.rows - 1)):
            raise Exception('Incorrect coordinates: either x should be 0 or y should be equal to: ', configuration.rows - 1)
        while x <= configuration.columns -1 and y >= 0:
            checker = get_checker_by_coord(x, y, observation, configuration)
            diag.append(checker)
            x += 1
            y += -1
        return diag
        
    #checks for a winner in an array. Returns 0 if none, 1 or 2 if there is a winner.
    def check_array_for_winner(array):
        winner = 0
        counter = 0
        prev_checker = 0
        for checker in array:
            if checker !=0 and checker == prev_checker:
                counter += 1
            else:
                counter = 0
            prev_checker = checker
            if counter == 3:
                return checker
        return winner
    
    #returns list of possible legal moves (column number)
    def list_of_possible_moves(observation, configuration):
        return [c for c in range(configuration.columns) if observation.board[c] == 0]    
    
    #makes a move given column number
    def simulate_move(x, observation, configuration):
        y = find_empty_row_in_column(x, observation, configuration)
        if y == -1:
            raise Exception('Illegal move! Column: ', y, ' has no empty spots!')
        observation = set_checker_by_coord(x, y, observation, configuration)
        return observation
    
    #sets checker to a value given board state and column
    def set_checker_by_coord(x, y, observation, configuration):
        current_mark = observation.mark
        observation.board[x + (configuration.columns * y)] = current_mark
        return observation
    
    #changes mark of the current player
    def change_current_player(observation, configuration):
        if observation.mark == 1:
            observation.mark = 2
        else:
            observation.mark = 1
        return observation
    
    # UNFINISHED----------------------------------------------------
    #simulates all possible legal moves, returns a winning move if possible, non winning moves for opponent on depth n=1 or a list of legal moves if any move leads to a loss
    def simulate_legal_moves(observation, configuration):
        moves = list_of_possible_moves(observation, configuration)
        observations = []
        winners = []
        for move in moves:
            temp_observation = deepcopy(observation)
            temp_observation = simulate_move(move, temp_observation, configuration)
            temp_observation = change_current_player(temp_observation, configuration)
            observations.append(temp_observation)
            winners.append(check_winner(temp_observation, configuration))
        return moves, observations, winners
            
    #returns 0 if board has no winner, 1 or 2 if there is only one, raises an exeption if both 1 and 2 are winners
    def check_winner(observation, configuration):
        winner_list = [0]
        
        #checking all columns for a winner
        for x in range(0, configuration.columns):
            winner = check_array_for_winner(get_column(x, observation, configuration))
            winner_list.append(winner)
        
        #checking all rows for a winner
        for y in range(0, configuration.rows):
            winner = check_array_for_winner(get_row(y, observation, configuration))
            winner_list.append(winner)
            
        #checking main diagonals for a winner
        top_row = [[x, 0] for x in  range(1, configuration.columns)]
        left_column = [[0, y] for y in range(0, configuration.rows)]
        for coord in (top_row+left_column):
            x = coord[0]
            y = coord[1]
            winner = check_array_for_winner(get_main_diagonal(x, y, observation, configuration))
            winner_list.append(winner)
        
        #checking secondary diagonals for a winner
        bottom_row = [[x, configuration.rows - 1] for x in  range(1, configuration.columns)]
        left_column = [[0, y] for y in range(0, configuration.rows)]
        for coord in (bottom_row+left_column):
            x = coord[0]
            y = coord[1]
            winner = check_array_for_winner(get_secondary_diagonal(x, y, observation, configuration))
            winner_list.append(winner)
            
        #get a winner 
        winner_list = np.unique(np.array(winner_list))
        if len(winner_list) == 3:
            raise Exception('More than two winners were found on the board state!')
        if len(winner_list) == 1:
            return 0
        return np.amax(winner_list)
        
    moves, new_observations, winners = simulate_legal_moves(observation, configuration)
    
    #checks for a winning move and returns it if need be
    for move, winner in zip(moves, winners):
        if winner == observation.mark:
            return move
    
    bad_moves = []
    for move, new_observation in zip(moves, new_observations):
        moves_depth2, new_observations_depth2, winners_depth2 = simulate_legal_moves(new_observation, configuration) 
        for winner_depth2 in winners_depth2:
            if winner_depth2 != observation.mark and winner_depth2 != 0:
                bad_moves.append(move)
                break
                
    potential_next_move = list_of_possible_moves(observation, configuration)[0]
    # Return which column to drop a checker (action).
    for x in list_of_possible_moves(observation, configuration):
        if x not in bad_moves:
            potential_next_move = x
    
    return potential_next_move